<a href="https://colab.research.google.com/github/jyporse/YOLO/blob/main/AppleCustomKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Google Drive Mount
from google.colab import drive
drive.mount("/content/Gdrive")

In [ ]:
!ls -la "/content/Gdrive/MyDrive"
# Check MyDrive

In [ ]:
# GPU 사용 (선택적으로 Tensorflow 의 Gpu 버전을 사용)
# 사용시 오류가 뜸 GPU 사용에대한 문제가 있는듯
!pip install tensorflow-gpu

In [ ]:
#Check Tensorflow Gpu
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print("Found GPU at: {}".format(device_name))
print(tf.__version__)

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Libraries
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import os
import cv2

# Define Constants
batch_size = 128
num_classes = 0
epochs = 200
CW = 32
CH = 32
CD = 3
model_name = 'apple_custom.h5'
tflite_model_name = "apple_custom.tflite"
COLAB_DARKNET_PATH = '/content/Gdrive/MyDrive/Kobot/YOLO/darknet'
YOLO_IMAGE_PATH = COLAB_DARKNET_PATH + '/images/'
YOLO_FORMAT_PATH = COLAB_DARKNET_PATH + '/images/'
classes = []

train_images = []
train_labels =[]
test_images = []
test_labels = []

sample_test_image = 'sample_test.jpg'
# Test를 위한 Sample 
sample_test_label = 0

In [ ]:
def show_sample(images, labels, sample_count = 25):
    grid_count = math.ceil(math.ceil(math.sqrt(sample_count)))
    grid_count = min(grid_count, len(images), len(labels))

    plt.figure(figsize=(2*grid_count, 2*grid_count))
    for i in range(sample_count):
        plt.subplot(grid_count, grid_count, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        image = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)
        plt.imshow(image, cmap=plt.cm.gray)
        plt.xlabel(labels[i])
    plt.show()

def download(path):
    try:
        from google.colab import files
        files.download(path)
    except ImportError:
        import os
        print("Error Download:", os.path.join(os.getcwd(), path))

def getROI(size, box):
    width_ratio = size[1]
    height_ratio = size[0]
    x = float(box[1]) * width_ratio
    y = float(box[2]) * height_ratio
    w = float(box[3]) * width_ratio
    h = float(box[4]) * height_ratio
    half_width = w/2.0
    half_height = h/2.0
    startX = int(x-half_width)
    startY = int(y-half_height)
    endX = int(x+half_width)
    endY = int(y+half_height)
    return (startY, endY, startX, endX)

def imShow(path):
    fig = plt.gcf()
    #fig.set_size_inches(18,10)
    plt.axis("off")
    plt.imshow(cv2.cvtColor(path, cv2.COLOR_BGR2RGB))
    plt.show()

YOLO의 classes를 채워주는 코드

In [ ]:
with open(YOLO_FORMAT_PATH+ 'classes.txt', 'r') as txt:
    for line in txt:
        name = line.replace("\n", "")
        classes.append(name)
        num_classes +=1
    print(classes, num_classes)

In [ ]:
## TRAIN
image_count = 0
with open(YOLO_FORMAT_PATH + 'train.txt', 'r') as txt:
    for line in txt:
        file_image_count = 0
        image_path = line.replace("\n","")
        img = cv2.imread(image_path)
        size = img.shape[:2]
        text_path = image_path[:-4] + '.txt'
        with open(text_path, 'r') as txt:
            for line in txt:
                box = line.split()
                (startY, endY, startX, endX) = getROI(size, box)
                image = cv2.resize(img[startY:endY , startX: endX], (CW,CH), interpolation = cv2.INTER_AREA)
                #image = cv2.cvtCOLOR(image, cv2.COLOR_BGR2RGB)
                #image = cv2.equalizeHist(image)
                #imShow(image)
                #print(int(box[0]))
                ## 위 주석코드는 오히려 인식률이 떨어진다.

                # Updata the list of data and labels, respectively

                train_images.append(image)
                train_labels.append(int(box[0]))
                image_count += 1
                file_image_count +=1
        print("%s : %d images added" % (image_path, file_image_count))
    # Convert the data and labels to Numpy arrays
    train_images = np.array(train_images)
    train_labels = np.array(train_labels)
    # Scalse data to the tange of [0,1]
    train_images = train_images.astype("float32") / 255.0
    # One-Hot encode the training and trsting labels
    train_labels = keras.utils.to_categorical(train_labels, num_classes)
    print('%d images added' % image_count)

In [ ]:
## TEST

image_count = 0
with open(YOLO_FORMAT_PATH + 'test.txt', 'r') as txt:
    for line in txt:
        file_image_count = 0
        image_path = line.replace("\n","")
        img = cv2.imread(image_path)
        size = img.shape[:2]
        text_path = image_path[:-4] + '.txt'
        with open(text_path, 'r') as txt:
            for line in txt:
                box = line.split()
                (startY, endY, startX, endX) = getROI(size, box)
                image = cv2.resize(img[startY:endY , startX: endX], (CW,CH), interpolation = cv2.INTER_AREA)
                #image = cv2.cvtCOLOR(image, cv2.COLOR_BGR2RGB)
                #image = cv2.equalizeHist(image)
                #imShow(image)
                #print(int(box[0]))
                ## 위 주석코드는 오히려 인식률이 떨어진다.
                
                # Updata the list of data and labels, respectively

                test_images.append(image)
                test_labels.append(int(box[0]))
                image_count += 1
                file_image_count +=1
        print("%s : %d images added" % (image_path, file_image_count))
    # Convert the data and labels to Numpy arrays
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)
    # Scalse data to the tange of [0,1]
    test_images = test_images.astype("float32") / 255.0
    # One-Hot encode the training and trsting labels
    test_labels = keras.utils.to_categorical(test_labels, num_classes)
    print('%d images added' % image_count)

In [ ]:
# Show images in the training dataset.

show_number = len(train_images)
print("Tatal number of Images : %d"  % show_number)

if(show_number >25):
    show_number = 25
show_sample(train_images, 
            [ '%s' % classes[np.argmax(label)] for label in train_labels], show_number)

In [ ]:
inputShape = (CH, CW, CD)

model = keras.Sequential([
                          keras.layers.Flatten(input_shape = inputShape),

                          keras.layers.Reshape(target_shape = inputShape),
                          keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation = tf.nn.relu),
                          keras.layers.Conv2D(filters=64, kernel_size =(3,3), activation=tf.nn.relu),
                          keras.layers.MaxPooling2D(pool_size =(2,2,)),
                          keras.layers.Dropout(0.25),
                          keras.layers.Flatten(input_shape=(28,28)),
                          keras.layers.Dense(128, activation = tf.nn.relu),
                          keras.layers.Dropout(0.5),

                          keras.layers.Dense(num_classes, activation= tf.nn.softmax)
])

model.compile(optimizer = 'adam',
              loss = tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
# Save the best model as digits_model.h5
modelCheckpoint = tf.keras.callbacks.ModelCheckpoint(model_name, save_best_only = True)

# Define a callback to monitor val_loss
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', 
                                                 patience =5)

In [ ]:
# Train the model usisng the early stopping callback
# earlyStopping을 이번실습에서는 제외한다.

history = model.fit(train_images, train_labels,
                    validation_data = (test_images, test_labels),
                    epochs = epochs, batch_size = batch_size,
                    callbacks=[modelCheckpoint])

In [ ]:
# Download the digit classification model if you're using Colab,
download(model_name)

In [ ]:
### Evaluate model


test_loss, test_acc = model.evaluate(test_images, test_labels)

print("Test Accuracy:", test_acc)

fig, loss_ax = plt.subplots()
gix, acc_ax = plt.subplots()

loss_ax.plot(history.history['loss'], 'ro', label = 'train loss')
loss_ax.plot(history.history['val_loss'], 'r:', label= 'val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['accuracy'], 'bp', label = 'train accuracy')
acc_ax.plot(history.history['val_accuracy'], 'b:', label= 'val accuracy')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

In [ ]:
## Predict the labels of digit images in our test dataset.

predictions = model.predict(test_images)
# Then plot the first 25 test images and their predicted labels.
show_number = len(test_images)
print('Tatal number of Images : %d' %show_number)
if(show_number > 25):
    show_number = 25
show_sample(test_images, 
            ['%s[%s]' % (classes[np.argmax(result)], np.argmax(result) == np.argmax(test_labels[idx]))
            for idx,result in enumerate(predictions)],
            show_number)


In [ ]:
# Predict Model
import random
ran = random.randint(0,25)
selected_digit = ran

result = predictions[selected_digit]
result_number = np.argmax(result)
label = test_labels[selected_digit]
label_number = np.argmax(label)
print("%s : %.2f [%s]" % (classes[result_number], result[result_number]*100, classes[label_number]))

imShow(test_images[selected_digit])